In [ ]:
!pip install gdown
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1Sg2Jn5Efmc8oT7WzhwK22SMKcmIbE3-Q

Downloading...
From: https://drive.google.com/uc?id=1Sg2Jn5Efmc8oT7WzhwK22SMKcmIbE3-Q
To: /content/dataset.zip
100% 23.3M/23.3M [00:00<00:00, 196MB/s]


In [ ]:
#melakukan extract zip
import zipfile
zip_file = zipfile.ZipFile('/content/dataset.zip')
zip_file.extractall('/content/latihan')
import os
try:
  os.mkdir('latihan')
except:
  print('path exist')



path exist


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

# Images Dimensions
img_width, img_height = 128, 128

train_data_dir = 'latihan/train'
validation_data_dir = 'latihan/validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

In [ ]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 800 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [ ]:
# Feature Extraction Layer
inputs = Input(shape=(img_width, img_height, 3))
conv_layer = Conv2D(16, (5, 5), strides=(3,3), activation='relu')(inputs) 
conv_layer = ZeroPadding2D(padding=(1,1))(conv_layer) 
conv_layer = Conv2D(32, (5, 5), strides=(3,3), activation='relu')(conv_layer) 
conv_layer = MaxPooling2D((2, 2))(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer) 
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer)

# Flatten Layer
flatten = Flatten()(conv_layer) 

# Fully Connected Layer
fc_layer = Dense(32, activation='relu')(flatten)
outputs = Dense(1, activation='sigmoid')(fc_layer)

model = Model(inputs=inputs, outputs=outputs)

# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

# Print Model Summary
print(model.summary())

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size, 
    callbacks=[callbacks])

model.save_weights('kornet.h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 42, 42, 16)        1216      
                                                                 
 zero_padding2d (ZeroPadding  (None, 44, 44, 16)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        12832     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 7, 32)         0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 64)          18496 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
<ipython-input-7-e8541938b081>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


None
Epoch 1/50
50/50 [==============================] - 13s 92ms/step - loss: 0.6919 - accuracy: 0.5387 - val_loss: 0.6904 - val_accuracy: 0.5500
Epoch 2/50
50/50 [==============================] - 4s 86ms/step - loss: 0.6863 - accuracy: 0.6000 - val_loss: 0.6841 - val_accuracy: 0.5958
Epoch 3/50
50/50 [==============================] - 4s 87ms/step - loss: 0.6723 - accuracy: 0.6438 - val_loss: 0.6724 - val_accuracy: 0.5542
Epoch 4/50
50/50 [==============================] - 4s 88ms/step - loss: 0.6458 - accuracy: 0.6325 - val_loss: 0.6589 - val_accuracy: 0.6125
Epoch 5/50
50/50 [==============================] - 4s 89ms/step - loss: 0.6273 - accuracy: 0.6538 - val_loss: 0.6547 - val_accuracy: 0.6000
Epoch 6/50
50/50 [==============================] - 4s 88ms/step - loss: 0.6076 - accuracy: 0.6675 - val_loss: 0.6350 - val_accuracy: 0.6208
Epoch 7/50
50/50 [==============================] - 4s 87ms/step - loss: 0.5888 - accuracy: 0.6988 - val_loss: 0.6299 - val_accuracy: 0.6333
Epoch 8

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

# Images Dimensions
img_width, img_height = 128, 128

train_data_dir = 'latihan/train'
validation_data_dir = 'latihan/validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

In [ ]:
# Build VGG16
model = applications.VGG16(include_top=False, weights='imagenet')

# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
train_features = model.predict_generator(
    train_generator, nb_train_samples // batch_size, verbose=1)
np.save('train_features.npy', train_features)

# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
validation_features = model.predict_generator(
    validation_generator, nb_train_samples // batch_size, verbose=1)
np.save('val_features.npy', validation_features)

58889256/58889256 [==============================] - 3s 0us/step
Found 800 images belonging to 2 classes.


<ipython-input-10-e047860642bc>:20: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  train_features = model.predict_generator(


50/50 [==============================] - 5s 77ms/step
Found 240 images belonging to 2 classes.
 3/50 [>.............................] - ETA: 1s

<ipython-input-10-e047860642bc>:30: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  validation_features = model.predict_generator(


14/50 [=======>......................] - ETA: 0s

50/50 [==============================] - 1s 12ms/step


In [ ]:
# Build Train Data
train_data = np.load('train_features.npy')
train_labels = np.array(
    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

# Build Validation Data
validation_data = np.load('val_features.npy')
validation_labels = np.array(
    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

# Build FC Layer
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(32, activation='relu'))
fc_model.add(Dense(1, activation='sigmoid'))

# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
fc_model.compile(optimizer=adam,
                loss='binary_crossentropy', metrics=['accuracy'])

fc_model.fit(train_data, train_labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels), 
            callbacks=[callbacks])
            
fc_model.save_weights('vgg16.py')

Epoch 1/50
50/50 [==============================] - 1s 11ms/step - loss: 0.7118 - accuracy: 0.5113 - val_loss: 0.7017 - val_accuracy: 0.4833
Epoch 2/50
50/50 [==============================] - 0s 4ms/step - loss: 0.6800 - accuracy: 0.5750 - val_loss: 0.7046 - val_accuracy: 0.5042
Epoch 3/50
50/50 [==============================] - 0s 3ms/step - loss: 0.6647 - accuracy: 0.6125 - val_loss: 0.7038 - val_accuracy: 0.5083
Epoch 4/50
50/50 [==============================] - 0s 4ms/step - loss: 0.6344 - accuracy: 0.6762 - val_loss: 0.7074 - val_accuracy: 0.4875
Epoch 5/50
50/50 [==============================] - 0s 4ms/step - loss: 0.6075 - accuracy: 0.6988 - val_loss: 0.7176 - val_accuracy: 0.4833
Epoch 6/50
50/50 [==============================] - 0s 4ms/step - loss: 0.5648 - accuracy: 0.7675 - val_loss: 0.7227 - val_accuracy: 0.4625
Epoch 7/50
50/50 [==============================] - 0s 3ms/step - loss: 0.5368 - accuracy: 0.7937 - val_loss: 0.7275 - val_accuracy: 0.5000
Epoch 8/50
50/50 [=

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam

img_width, img_height = 128, 128

test_data_dir = 'latihan/testing'

def get_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
    features = model.predict_generator(
        generator, 8)
    return features

def get_score():
    features = get_features()    

    model = Sequential()
    model.add(Flatten(input_shape=features.shape[1:]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    model.load_weights('vgg16.py')

    score = model.predict(features)
    print(score)

get_score()

Found 8 images belonging to 1 classes.


<ipython-input-14-0965a2535447>:24: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  features = model.predict_generator(


1/1 [==============================] - 0s 37ms/step
[[0.8611809 ]
 [0.02038782]
 [0.03804415]
 [0.1958484 ]
 [0.83914244]
 [0.11303189]
 [0.5609708 ]
 [0.60171515]]
